In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import random
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from torch import optim
import time
import math
import os
use_cuda = torch.cuda.is_available()
# device = torch.device("cpu")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


Сгенерировать меньший датасет из 8-10 классов движения


In [2]:
data_path = "nturgb+d_skeletons/"
#### список отсутсвующих элементов так же будет доступен 
broken_files_path = "NTU_RGBD_samples_with_missing_skeletons.txt"

In [3]:
training_subjects = list(range(0, 28)) #количество людей выполняющих действия
training_classes = [1,2,3,4,5,6,7,8,9,10] #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
training_cameras = [1, 2, 3] 

num_joint = 25
max_frame = 300 # Длина отрезка которую мы вычленяем из большого датасета

In [4]:
LABELS={0 :'drink water', 1 :'eat meal/snack', 2 :' brushing teeth', 
        3 :'brushing hair', 4 :'drop', 5 :'pickup', 6 : 'throw',
        7 : 'sitting down', 8 : 'standing up (from sitting position)',
        9 : 'clapping'
       }

In [5]:
def read_data(data_path, broken_files_path):
    labels = []
    files = []
    action_classes = {}
    counter = 0
    files_counter = {}
              
    with open(broken_files_path, 'r') as f:
        broken_files = f.read().split("\n")

    raw_files = os.listdir(data_path)
    num_frames = 0

    for filename in raw_files:
        if filename not in broken_files:
            action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
            subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
            camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
            if action_class in training_classes and camera_id in training_cameras:  #and subject_id in training_subjects:
                if action_class in action_classes:
                    if files_counter[action_class] < 120:
                        files.append([filename,action_classes[action_class]])
                        files_counter[action_class] = files_counter[action_class] + 1
                else:
                    action_classes.update({action_class : counter})
                    files_counter.update({action_class : 1})
                    counter+=1
                    files.append([filename,action_classes[action_class]])
#                     labels.append([action_class])
#     print("action classes: ", action_classes)
#     print("action files: ", files_counter)
    
    return files, action_classes

In [6]:
# Функция фильтр для того, что бы найти только координаты x,y,z(т.к. в датасете
# хранится на порядок больше информации, нежели только координаты x,y,z)
# (Остальные данные нам и не нужны, т.к. у нас нет ик-камер)
def read_skeleton_filter(file):
    with open(file, 'r') as f:
        skeleton_sequence = {}
        skeleton_sequence['numFrame'] = int(f.readline())
        skeleton_sequence['frameInfo'] = []
        for t in range(skeleton_sequence['numFrame']):
            frame_info = {}
            frame_info['numBody'] = int(f.readline())
            frame_info['bodyInfo'] = []

            for m in range(frame_info['numBody']):
                body_info = {}
                body_info_key = [
                    'bodyID', 'clipedEdges', 'handLeftConfidence',
                    'handLeftState', 'handRightConfidence', 'handRightState',
                    'isResticted', 'leanX', 'leanY', 'trackingState'
                ]
                body_info = {
                    k: float(v)
                    for k, v in zip(body_info_key, f.readline().split())
                }
                body_info['numJoint'] = int(f.readline())
                body_info['jointInfo'] = []
                for v in range(body_info['numJoint']):
                    joint_info_key = [
                        'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                        'orientationW', 'orientationX', 'orientationY',
                        'orientationZ', 'trackingState'
                    ]
                    joint_info = {
                        k: float(v)
                        for k, v in zip(joint_info_key, f.readline().split())
                    }
                    body_info['jointInfo'].append(joint_info)
                frame_info['bodyInfo'].append(body_info)
            skeleton_sequence['frameInfo'].append(frame_info)

    return skeleton_sequence

# Здесь мы используем нашу функцию фильр и оформляем дату в x,y,z-cкоординаты
def read_xyz(file, max_body=1, num_joint=25):
    seq_info = read_skeleton_filter(file)
    data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
    for n, f in enumerate(seq_info['frameInfo']):
        for m, b in enumerate(f['bodyInfo']):
            for j, v in enumerate(b['jointInfo']):
                if m < max_body and j < num_joint:
                    data[m, n, j, :] = [v['x'], v['y'], v['z']]

                else:
                    pass

    return data

In [7]:
def create_coords_blocks(test_file, chonk_len = 45):   
    frame_counter = 0
    new_labels = []
    new_frames = []
    blocks = []
    
    test_frames = read_xyz(data_path + test_file[0])[0]
#     print(test_frames.shape)
    label = test_file[1]
    slice_len = chonk_len * int(len(test_frames)/chonk_len)


    for index in range(len(test_frames[:slice_len])):
        frame_counter += 1
        new_frames.append(test_frames[index].flatten())
        if frame_counter == chonk_len:
            frame_counter = 0
            blocks.append(np.array(new_frames))
#             print(np.array(new_frames).shape)
            new_labels = new_labels + [label]
            new_frames = []
#     print(new_labels,len(blocks))   
            
    return blocks, new_labels

In [8]:
##### список файлов с лейблами на каждый файл 


### Здесь выносится сгенерированный список лейблов с номером класса.

In [9]:
def transform_(chonk_len):
    
    working_files_with_labels, action_classes = read_data(data_path, broken_files_path)
    data = []
    labels = []
    numbers = {x: 0 for x in range(len(action_classes))}
    for file in working_files_with_labels:
        frames_blocks, label = create_coords_blocks(file, chonk_len)
        if label != [] and numbers[label[0]] <= 150:
    #         print(numbers[label[0]],len(label))
            numbers[label[0]] = numbers[label[0]] + len(label)

            data = data + frames_blocks
            labels = labels + label
    #         print(labels)
    data_np = np.asarray(data)
    labels_np = np.asarray(labels)

    data_sq = data_np.reshape(len(data_np), -1)
    test_data = pd.DataFrame(data_sq)
    test_labels = pd.DataFrame(labels_np)
    test_data['labels'] = test_labels
    
    return test_data.to_csv("skeletons_classes_1_30.csv", index = False)

In [10]:
class LSTM_net(nn.Module):
    # Для того, что бы инициализировать LSTM нам нужно указать:
    # input_dim - размерность входного тензора. тензор входит в формате (seq_len, batch, input_dim)
    # (batch_size, seq, inp_dim) - if batch_first=True
    # hidden_dim - размерность вектора состояния h
    # output_dim - размерность выхода
    # layer_num - количество скрытых слоев в сети
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [44]:
class Skeleton_Dataset(Dataset):
    def __init__(self, file_path, transform=None,chonk_len=45):
        self.data = pd.read_csv(file_path)
        self.transform = transform(chonk_len)
        self.labels = self.data.iloc[:,-1]
        self.chonk_len=chonk_len


    def __len__(self):
        return len(self.data) 
    
    
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx,:-1]).reshape(45,25*3)
        label = self.labels[idx]
        if self.transform != None:
            item = transform(item)
            
        

        return (item, label)

In [13]:
dataset = Skeleton_Dataset(file_path = "skels.csv", transform=transform_,chonk_len=60)

In [14]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.75*len(dataset)),int(0.25*len(dataset))])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=True)

In [50]:
def make_data(file_path = "skels.csv", transform=transform_,chonk_len=45) :
    dataset = Skeleton_Dataset(file_path = "skels.csv", transform=transform_,chonk_len=45)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.75*len(dataset)),int(0.25*len(dataset))])
    train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=True)
    
    return train_loader, test_loader



In [51]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
#     print(output.topk(5))
    return LABELS[category_i], category_i

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [52]:
def train_model(n_hidden = 128,n_joints = 25*3, 
                n_categories = len(LABELS),
               n_layer = 2, epochs=300, seed=101,
               learning_rate = 0.009):


    rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
    rnn.to(device)

    torch.cuda.manual_seed_all(seed)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, learning_rate, epochs=epochs, 
                                                    steps_per_epoch=len(train_loader))

    all_losses = []
    start = time.time()
    counter = 0

    # torch.save(rnn.state_dict(), "model.pth")
    # rnn.load_state_dict(torch.load("model.pth"))
    # optimizer.param_groups[0]['lr'] = 0.009

    for epoch in range(epochs):  
        current_loss = 0
        running_loss = 0.0

    #     if epoch == 600:
    #                 optimizer.param_groups[0]['lr'] = 0.005

        for i, data in enumerate(train_loader, 0):

            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()


            output = rnn(inputs.float())
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()


            current_loss += loss.item()
            category = LABELS[int(labels[0])]

            
    return rnn

In [53]:
def make_eval(rnn, param=''):

    total = 0
    right = 0
    counter = 0

    rnn.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            counter = counter + 1
            inputs, labels = data[0].to(device), data[1].to(device)  
            output = rnn(inputs.float())
            guess, guess_i = categoryFromOutput(output)
            category = LABELS[int(labels[0])]

            if guess == category:
                right = right + 1


    print('param= ', param, 'Accuracy of the network:  ',  (100 * right / counter))


Обучить уже существующую модель (предварительно проанализировав какие параметры модели нужно изменить)


In [54]:
train_loader, test_loader= make_data()

In [46]:
epochs=[100,300,600,1000]

for i in epochs:
          
    rnn=train_model(epochs=i)
    make_eval(rnn, i)

param=  100 Accuracy of the network:   81.13207547169812
param=  300 Accuracy of the network:   86.79245283018868
param=  600 Accuracy of the network:   89.30817610062893
param=  1000 Accuracy of the network:   87.42138364779875


In [47]:
learning_rate=[0.013,0.009,0.005,0.001]

for i in learning_rate:
           
    rnn=train_model(epochs=600, learning_rate=i)
    make_eval(rnn, i)

param=  0.013 Accuracy of the network:   84.90566037735849
param=  0.009 Accuracy of the network:   88.0503144654088
param=  0.005 Accuracy of the network:   86.79245283018868
param=  0.001 Accuracy of the network:   83.01886792452831


In [43]:
n_layer=[2, 3, 4]

for i in n_layer:
       
    rnn=train_model(n_layer=i)
    make_eval(rnn, i)

param=  2 Accuracy of the network:   90.56603773584905
param=  3 Accuracy of the network:   84.27672955974843
param=  4 Accuracy of the network:   82.38993710691824


In [55]:
rnn=train_model(learning_rate=0.009,epochs=600)
make_eval(rnn)

param=   Accuracy of the network:   91.19496855345912


Изменение n_hidden не оказывет существенного влияния на модель.
Оптимайзеры sgd и adam дают схожие результаты, rmsprop показывает хуже результат.
Значительное влияние на итоговой результат оказывает неопределенность модели и 
cuda.manual_seed не сильно меняет ситуацию. Разница в результатах при повторных запусках
может достигать 5-10%,  сохранение и загрузка параметров инициалированной модели также
не избавляет от плавающей метрики.
Выход при оценки модели видится в создании ансамблей из повторных запусков модели, но это требует много времени.
Можно попробывать инициализировать веса каждого слоя, но я с этим не разбирался еще.

Изменить модель: посмотреть зависимость от количества LSTM модулей в нашей модели  


In [ ]:
# Увеличение количества слоев LSTM ухудшает качество медели. Воспроизводить не буду, потому что долго тренеруется.

Сгенерировать другой датасет с меньшим количеством “кадров” в серии и сравнить улучшилось или ухудшилось качество предсказания. Провести несколько таких итераций, дать свою оценку уменьшению и увеличению кадров, назвать оптимальное, на ваш взгляд, их количество. Желательно сделать так, чтобы длина последовательности передавалась как атрибут класса.


In [32]:
chonk_lens=[15,25,45,65,85]

for i in chonk_lens:
    
     
    train_loader, test_loader= make_data(chonk_len=i)
    rnn=train_model()
    make_eval(rnn, i)

param=  15 Accuracy of the network:   88.67924528301887
param=  25 Accuracy of the network:   84.90566037735849
param=  45 Accuracy of the network:   89.937106918239
param=  65 Accuracy of the network:   86.79245283018868
param=  85 Accuracy of the network:   86.79245283018868
